In [4]:
import matplotlib as plt
from matplotlib import pyplot
import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import os
import pywt
from scipy import signal
import math
from math import *
from numpy import *
import sys
from biosppy.signals import ecg
import statistics

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pywt

from sklearn.metrics import classification_report

from keras.models import load_model

In [ ]:
# Load model
#model = load_model('Models/best_model.08-0.17.h5')
model = load_model('Models/best_model.01-0.04.h5') #Insert model here
print('Model loaded')

Model loaded


In [38]:
# Load target dataset
data = pd.read_csv("PTB_processed_new/record333_patient121_Healthy control.csv", header = 0)

In [39]:
# Process data to be similar to input training data
signal_array = []
file_array = []
for column in data:
    if column != 'vx' and column != 'vy' and column != 'vz':
        df = data[column]
        arrays = np.array(df)
        arrays = baseline_corr(arrays)
        peak = signal.argrelmax(arrays, order = 450)
        start = peak[0][2] - 200
        end = peak[0][2] + 400
        arrays = arrays[start:end]
        arrays = wavelet_denoise(arrays)
        arrays = signal.resample(arrays, 150)
        file_array.append(arrays)

signal_array.append(file_array)
signal_array = np.asarray(signal_array).reshape(-1, 150, 12)

print ('Input Loaded')

Input Loaded


In [40]:
# Process data to be similar to input training data
signal_array = []
file_array = []

#data = baseline.baseline_corr(data)
#data = wden.wdenoise(data, 'sqtwolog', 'soft', 'mln', 4, 'sym7')
df = np.array(data['v4'])
    
peaks = ecg.christov_segmenter(df, 250)[0]
for peak in range (1, len(peaks) - 1):
    segment_array = []
    for column in data:
        df = np.array(data[column])
        segment = df[peaks[peak]-50:peaks[peak]+100]
        segment_array.append(segment)  
    signal_array.append(segment_array)

signal_array = np.asarray(signal_array).reshape(-1, 150, 12)

print ('Input Loaded')
print (signal_array.shape[0])

Input Loaded
41


In [ ]:
# Transform data into 12 sections (Only use if the rest breaks)
input1 = signal_array[:, :, 0].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input2 = signal_array[:, :, 1].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input3 = signal_array[:, :, 2].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input4 = signal_array[:, :, 3].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input5 = signal_array[:, :, 4].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input6 = signal_array[:, :, 5].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input7 = signal_array[:, :, 6].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input8 = signal_array[:, :, 7].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input9 = signal_array[:, :, 8].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input10 = signal_array[:, :, 9].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input11 = signal_array[:, :, 10].reshape(signal_array.shape[0], signal_array.shape[1], 1)
input12 = signal_array[:, :, 11].reshape(signal_array.shape[0], signal_array.shape[1], 1)

In [ ]:
# Predictor (Only use if the rest breaks)
y_pred_test = model.predict([input1, input2, input3, input4, input5, input6, input7, input8, input9, input10, input11, input12], verbose = 1)
# Take the class with the highest probability from the test predictions
prediction = np.argmax(y_pred_test, axis=1)

labels = ['Bundle branch block', 'Cardiomyopathy', 'Dysrhythmia', 'Healthy control', 'Hypertrophy', 'Myocardial infarction', 'Myocarditis', 'Stable angina', 'Valvular heart disease']

print (labels[int(prediction)])

In [43]:
predictions = []
for i in range (signal_array.shape[0]):
    input1 = signal_array[i, :, 0].reshape(1, signal_array.shape[1], 1)
    input2 = signal_array[i, :, 1].reshape(1, signal_array.shape[1], 1)
    input3 = signal_array[i, :, 2].reshape(1, signal_array.shape[1], 1)
    input4 = signal_array[i, :, 3].reshape(1, signal_array.shape[1], 1)
    input5 = signal_array[i, :, 4].reshape(1, signal_array.shape[1], 1)
    input6 = signal_array[i, :, 5].reshape(1, signal_array.shape[1], 1)
    input7 = signal_array[i, :, 6].reshape(1, signal_array.shape[1], 1)
    input8 = signal_array[i, :, 7].reshape(1, signal_array.shape[1], 1)
    input9 = signal_array[i, :, 8].reshape(1, signal_array.shape[1], 1)
    input10 = signal_array[i, :, 9].reshape(1, signal_array.shape[1], 1)
    input11 = signal_array[i, :, 10].reshape(1, signal_array.shape[1], 1)
    input12 = signal_array[i, :, 11].reshape(1, signal_array.shape[1], 1)
    
    # Predictor
    y_pred_test = model.predict([input1, input2, input3, input4, input5, input6, input7, input8, input9, input10, input11, input12], verbose = 1)
    # Take the class with the highest probability from the test predictions
    prediction = np.argmax(y_pred_test, axis=1)
    
    labels = ['Bundle branch block', 'Cardiomyopathy', 'Dysrhythmia', 'Healthy control', 'Hypertrophy', 'Myocardial infarction', 'Myocarditis' 'Stable angina'
 'Valvular heart disease']
    predictions.append(labels[int(prediction)])
    #print (labels[int(prediction)])
    

final_prediction = statistics.mode(predictions) 
print (final_prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step

C:\Users\tansh\AppData\Local\Temp\ipykernel_2512\3731628915.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  predictions.append(labels[int(prediction)])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

In [11]:
print (signal_array.shape)

(1, 150, 12)


In [9]:
def wavelet_denoise(data, wavelet='sym7', level=4, mode='soft'): #Update
    coeffs = pywt.wavedec(data, wavelet, level=level)
    threshold = np.sqrt(2 * np.log(len(data))) * np.median(np.abs(coeffs[-level])) / 0.6745
    coeffs[1:] = [pywt.threshold(c, threshold, mode=mode) for c in coeffs[1:]]
    return pywt.waverec(coeffs, wavelet)

In [10]:
def baseline_corr(array):
    # Function for use in baseline removal
    def wrcoef(X, coef_type, coef, wavename, level):
        N = np.array(X).size
        a, ds = coef[0], list(reversed(coef[1:]))

        if coef_type =='a':
            return pywt.upcoef('a', a, wavename, level=level)[:N]
        elif coef_type == 'd':
            return pywt.upcoef('d', ds[level-1], wavename, level=level)[:N]
        else:
            raise ValueError("Invalid coefficient type: {}".format(coef_type))
    #Baseline correction variables
    wavename = 'db1'
    #Baseline correction
    coef = pywt.wavedec(array, wavename, level=10)
    A10 = wrcoef(array, 'a', coef, wavename, 10)
    D10 = wrcoef(array, 'd', coef, wavename, 10)
    D9 = wrcoef(array, 'd', coef, wavename, 9)
    D8 = wrcoef(array, 'd', coef, wavename, 8)
    D7 = wrcoef(array, 'd', coef, wavename, 7)
    D6 = wrcoef(array, 'd', coef, wavename, 6)
    D5 = wrcoef(array, 'd', coef, wavename, 5)
    D4 = wrcoef(array, 'd', coef, wavename, 4)
    D3 = wrcoef(array, 'd', coef, wavename, 3)
    D2 = wrcoef(array, 'd', coef, wavename, 2)
    D1 = wrcoef(array, 'd', coef, wavename, 1)
    array = D10 + D9 + D8 + D7 + D6 + D5 + D4 + D3 + D2 + D1
    return array